# Pre-Experiment
Define All Variables to Run the Experiment
    <p>```setup.bash``` Exports the Variables to a ```setup.conf``` and sends it to the server</p>

In [1]:
username=yousifd # Username of user on server
exp_name=ye # Name of Experiment on NOME
proj_name=cops # Name of Project on NOME
default_node=h0 # Default node to run the experiment from
ds=cops2
host_ip=nome.nmc-probe.org
default_ip=${default_node}.${exp_name}.${proj_name}

# Setup Number of Servers and Clients
nservers=8 # Number of Servers and Clients
ndcs=2 # Number of Data Centers
eindex=0
ipprefix=h
ippostfix="-dib"
threads=1
replication_factor=$ndcs

# Initialize Consistent Variables
trial=1 # Number of Trials in the Experiment
nkeys=100000 # Number of keys for the Experiment
run_time=30 # Time to run the Expierment
trim=5 # Trimmed time from the gathered results

# Initialize Default Variables
value_size=128 # Size of the Value Read and/or Written
cols_per_key_read=5 # Columns per key read
cols_per_key_write=5 # Columns per key write
keys_per_read=5 # Keys per read
keys_per_write=5 # Keys per write
write_frac="0.1" # Fraction of Write Operations to Read Operations
write_trans_frac="0.5" # Fraction of Write Transcation to Read Transactions

# Specify the Experiment's Indepdent Variable and its Values
exp=columns_per_read # Name of the Experiment
indep_values="1:${nkeys}" # Independen Variables to be Experimented with

insert_cmd=INSERTCL
KILLALL_SSH_TIME=90
MAX_ATTEMPTS=10

alias server="ssh ${username}@${host_ip} -o StrictHostKeyChecking=no ssh ${default_ip} -t -t -o StrictHostKeyChecking=no"

source /home/yousifd/setup.bash

setup.conf                                    100%  419     0.4KB/s   00:00    
dynamic_common                                100% 6460     6.3KB/s   00:00    
kodiak_dc_launcher.bash                       100% 4801     4.7KB/s   00:00    


# Experiment
Launch Instances of the Distributed System
    
Setup Benchmark on Clients
    
Start the Experiment
    
Kill all Instances of the Distributed System

In [2]:
server <<- 'EOF'

source dynamic_common

init

for t in $(seq $trial); do
for i in $(seq 0 $((${#var[@]} - 1))); do
    
    setup

    launch
    
    set_server_keyspace
    
    set_client_keyspace
    
    for i in $(seq 1 19); do
        ssh -o StrictHostKeyChecking=no h${i}-dib "collectl -P -ou -R $((run_time+20))s -f/tmp/${ds}_${t}" &
    done
    
    run_experiment
    
    kill_all_ds
    
    gather_results
    
    for i in $(seq 1 19); do
        rsync --remove-source-files -az h${i}-dib:/tmp/${ds}* ${output_dir}
    done
    
done; done; exit; exit;
EOF

rsync -az --copy-links ${username}@${host_ip}:/users/${username}/${ds}/experiments/dynamic_${exp}/latest /home/$USER

cd /home/$USER/latest
mkdir data
for i in $(ls *.gz); do
    mv ${i} ./data/
    gunzip ./data/${i}
done

















































































































































































































































































































































































































mkdir: cannot create directory 'data': File exists
ls: cannot access *.gz: No such file or directory


# Data Gathering and Post Processing
Gather Data from Servers and Clients
    
Post Process the Data into Meaningful Chunks
    
Generate Graphs from the Data Chuncks

In [3]:
output_dir=/home/$USER/latest

source setup.conf

cd /home/$USER/postprocess
./dynamic_postprocess_full.bash ${output_dir} ${output_dir} ${run_time} ${trim} shuffle; exit; exit;

client0
  cops2/
    trial1/
client1
  cops2/
    trial1/
client2
  cops2/
    trial1/
client3
  cops2/
    trial1/
mkdir: cannot create directory '.trash': File exists
mv: cannot move 'client0' to '.trash/client0': Directory not empty
mv: cannot move 'client1' to '.trash/client1': Directory not empty
mv: cannot move 'client2' to '.trash/client2': Directory not empty
mv: cannot move 'client3' to '.trash/client3': Directory not empty
/home/yousifd/latest/client0/
 cops2
  Client Postprocess: complete     
ls: cannot access /home/yousifd/latest/client0//cops2//client0/*+tput: No such file or directory
  Combining Clients: complete      
/home/yousifd/latest/client1/
 cops2
  Client Postprocess: complete     
ls: cannot access /home/yousifd/latest/client1//cops2//client0/*+tput: No such file or directory
  Combining Clients: complete      
/home/yousifd/latest/client2/
 cops2
  Client Postprocess: complete     
ls: cannot access /home/yousifd/latest/client2//cops2//client0/*+tput: No such

# Graphing

In [4]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
%cd /home/yousifd/data
plt.plotfile('cops2.graph.keys', delimiter=' ', cols=(0, 1), names=('indep', 'keys'), marker='o')
plt.plotfile('cops2.graph.cols', delimiter=' ', cols=(0, 1), names=('indep', 'cols'), marker='o')
plt.plotfile('cops2.graph.ops', delimiter=' ', cols=(0, 1), names=('indep', 'ops'), marker='o')
plt.plotfile('cops2.graph.bytes', delimiter=' ', cols=(0, 1), names=('indep', 'bytes'), marker='o')

bash: fg: %matplotlib: no such job
The program 'import' can be found in the following packages:
 * imagemagick
 * graphicsmagick-imagemagick-compat
Try: sudo apt-get install <selected package>
The program 'import' can be found in the following packages:
 * imagemagick
 * graphicsmagick-imagemagick-compat
Try: sudo apt-get install <selected package>
bash: fg: %cd: no such job
bash: syntax error near unexpected token `'cops2.graph.keys','
bash: syntax error near unexpected token `'cops2.graph.cols','
bash: syntax error near unexpected token `'cops2.graph.ops','
bash: syntax error near unexpected token `'cops2.graph.bytes','
